# 1. What is a SQL Endpoint in Microsoft Fabric?

A SQL Endpoint is a read-only T-SQL interface that automatically appears when you create a Lakehouse in Microsoft Fabric.
It lets you run Transact-SQL queries against the structured data stored in your Lakehouse without having to write Spark code or move the data into a database.

Think of it as:
    Your Lakehouse has two doors —

1️⃣ A Spark door for data engineers (PySpark, notebooks)

2️⃣ A SQL door for analysts & BI tools (SQL queries, SSMS, Power BI)

# 2. Where SQL Endpoints Fit in Fabric Architecture

Microsoft Fabric has a unified data platform with multiple experiences:

- Lakehouse → stores raw + curated data in Delta format.

- Data Warehouse → SQL-based tables with transactional capabilities.

- Notebooks / Dataflows → data transformation.

- Power BI → data visualization.

The SQL Endpoint sits between the Lakehouse and SQL-based consumers.
It translates Delta tables in your Lakehouse into a virtual SQL schema exposed via TDS protocol (the same as SQL Server).

📍 Key difference from Warehouse:

- Warehouse = stores & manages data inside a relational model (read/write).

- SQL Endpoint = read-only query layer over Delta tables in a Lakehouse.

# 3. How SQL Endpoints Work
Step-by-step:

1. Create a Lakehouse in Microsoft Fabric.

2. Load data into the Lakehouse — either into the Tables folder (structured) or Files folder (raw files).

3. When data is in the Tables folder (Delta tables), Fabric automatically generates a SQL schema.

4. A SQL Endpoint URL is created for your Lakehouse.

5. You can connect to this endpoint from:

- Power BI (DirectQuery or import mode)

- Excel (Get Data → SQL Server)

- SQL Server Management Studio (SSMS)

- Azure Data Studio

- Any JDBC/ODBC SQL client.

6. You query the data using standard T-SQL — no Spark knowledge required.

# 4. Key Features
| Feature                 | Description                                                     |
| ----------------------- | --------------------------------------------------------------- |
| **Read-only access**    | No `INSERT`, `UPDATE`, `DELETE`. Only `SELECT`.                 |
| **T-SQL support**       | SELECT, JOIN, GROUP BY, ORDER BY, Window functions, etc.        |
| **Auto schema sync**    | New tables in *Tables* folder instantly appear in SQL Endpoint. |
| **Delta table backend** | Reads directly from Delta tables stored in OneLake.             |
| **Connectivity**        | TDS protocol — compatible with SQL Server tools.                |
| **Security**            | Microsoft Entra ID authentication + Fabric workspace RBAC.      |
| **Power BI ready**      | DirectQuery supported for real-time analysis.                   |


# 5. Example Query via SQL Endpoint

If your Lakehouse has this structure:

Tables/

 ├── Customers

 ├── Sales
 
 └── Products

 You could run:

SELECT c.CustomerName, SUM(s.Amount) AS TotalSpent
FROM Sales s
JOIN Customers c ON s.CustomerID = c.CustomerID
GROUP BY c.CustomerName
ORDER BY TotalSpent DESC;

✅ Runs directly on Delta tables
✅ No ETL required



# 6. Connection Details

| Parameter          | Value                                                          |
| ------------------ | -------------------------------------------------------------- |
| **Server Name**    | Provided in the "SQL Endpoint" tab of your Lakehouse (TDS URL) |
| **Authentication** | Microsoft Entra ID                                             |
| **Protocol**       | TDS (Tabular Data Stream)                                      |
| **Port**           | 1433                                                           |
| **Tools**          | Power BI, Excel, SSMS, Azure Data Studio, JDBC/ODBC            |


# 7. Performance Considerations

- Use optimized Delta tables (Z-Ordering, vacuuming small files) for faster reads.

- Partition tables for large datasets to improve query pushdown.

- Use filter predicates in SQL to avoid scanning unnecessary data.

- Avoid Cartesian joins (performance hit).

- Large aggregations may be slower compared to a dedicated Warehouse because it’s a virtual read layer.

# 8. Limitations

- No DML: No INSERT, UPDATE, DELETE.

- No DDL: You can’t create or drop tables via SQL Endpoint.

- Tables only: Only data in Tables folder is exposed (Files folder is ignored).

- No stored procedures, triggers, functions.

- Case sensitivity: Table and column names can be case-sensitive depending on Delta configuration.

# 9. Security

- Access is controlled at the Fabric workspace level.

- Users need at least Viewer role in the workspace to query via SQL Endpoint.

- All queries require Microsoft Entra ID sign-in — no SQL logins.

- Row-level or column-level security must be implemented at the Power BI or Warehouse level, not inside the SQL Endpoint.

# 10. Common Use Cases

1. BI Integration — Power BI DirectQuery over Lakehouse tables.

2. Ad-hoc SQL analysis — Analysts can query with T-SQL instead of learning Spark.

3. Data validation — Compare Lakehouse data to external sources via SSMS.

4. Prototyping dashboards — Quickly connect Excel to Lakehouse data.

5. Federated querying — Join Lakehouse data with Warehouse or external SQL sources in Power BI.

💡 Pro Tip: If you need write capabilities or complex SQL features, you should move the data into a Fabric Data Warehouse. But for quick analysis and BI connections — SQL Endpoint is faster and simpler.